# 07 | 文本聚类与摘要，让 AI 帮你做个总结

## 基于 Embedding 向量进行文本聚类

In [1]:

from sklearn.datasets import fetch_20newsgroups
import pandas as pd

def twenty_newsgroup_to_csv():
    newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

    df = pd.DataFrame([newsgroups_train.data, newsgroups_train.target.tolist()]).T
    df.columns = ['text', 'target']

    targets = pd.DataFrame( newsgroups_train.target_names, columns=['title'])

    out = pd.merge(df, targets, left_on='target', right_index=True)
    out.to_csv('20_newsgroup.csv', index=False)
    
twenty_newsgroup_to_csv()

In [3]:

from openai.embeddings_utils import get_embeddings
import openai, os, tiktoken, backoff

openai.api_key = os.environ.get("OPENAI_API_KEY")
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
batch_size = 2000
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

df = pd.read_csv('20_newsgroup.csv')
print("Number of rows before null filtering:", len(df))
df = df[df['text'].isnull() == False]
encoding = tiktoken.get_encoding(embedding_encoding)

df["n_tokens"] = df.text.apply(lambda x: len(encoding.encode(x)))
print("Number of rows before token number filtering:", len(df))
df = df[df.n_tokens <= max_tokens]
print("Number of rows data used:", len(df))

Number of rows before null filtering: 11314
Number of rows before token number filtering: 11096
Number of rows data used: 11044


In [5]:

@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def get_embeddings_with_backoff(prompts, engine):
    embeddings = []
    for i in range(0, len(prompts), batch_size):
        batch = prompts[i:i+batch_size]
        embeddings += get_embeddings(list_of_text=batch, engine=engine)
    return embeddings

prompts = df.text.tolist()
prompt_batches = [prompts[i:i+batch_size] for i in range(0, len(prompts), batch_size)]

embeddings = []
for batch in prompt_batches:
    batch_embeddings = get_embeddings_with_backoff(prompts=batch, engine=embedding_model)
    embeddings += batch_embeddings

df["embedding"] = embeddings
df.to_parquet("data/20_newsgroup_with_embedding.parquet", index=False)

In [6]:

import numpy as np
from sklearn.cluster import KMeans

embedding_df = pd.read_parquet("data/20_newsgroup_with_embedding.parquet")

matrix = np.vstack(embedding_df.embedding.values)
num_of_clusters = 20

kmeans = KMeans(n_clusters=num_of_clusters, init="k-means++", n_init=10, random_state=42)
kmeans.fit(matrix)
labels = kmeans.labels_
embedding_df["cluster"] = labels

In [7]:

# 统计每个cluster的数量
new_df = embedding_df.groupby('cluster')['cluster'].count().reset_index(name='count')

# 统计这个cluster里最多的分类的数量
title_count = embedding_df.groupby(['cluster', 'title']).size().reset_index(name='title_count')
first_titles = title_count.groupby('cluster').apply(lambda x: x.nlargest(1, columns=['title_count']))
first_titles = first_titles.reset_index(drop=True)
new_df = pd.merge(new_df, first_titles[['cluster', 'title', 'title_count']], on='cluster', how='left')
new_df = new_df.rename(columns={'title': 'rank1', 'title_count': 'rank1_count'})

# 统计这个cluster里第二多的分类的数量
second_titles = title_count[~title_count['title'].isin(first_titles['title'])]
second_titles = second_titles.groupby('cluster').apply(lambda x: x.nlargest(1, columns=['title_count']))
second_titles = second_titles.reset_index(drop=True)
new_df = pd.merge(new_df, second_titles[['cluster', 'title', 'title_count']], on='cluster', how='left')
new_df = new_df.rename(columns={'title': 'rank2', 'title_count': 'rank2_count'})
new_df['first_percentage'] = (new_df['rank1_count'] / new_df['count']).map(lambda x: '{:.2%}'.format(x))
# 将缺失值替换为 0
new_df.fillna(0, inplace=True)
# 输出结果
display(new_df)

,cluster,count,rank1,rank1_count,rank2,rank2_count,first_percentage
0,0,481,sci.electronics,338,comp.sys.mac.hardware,27.0,70.27%
1,1,1049,comp.sys.ibm.pc.hardware,404,comp.sys.mac.hardware,388.0,38.51%
2,2,639,talk.politics.misc,304,talk.religion.misc,20.0,47.57%
3,3,493,rec.sport.baseball,476,0,0.0,96.55%
4,4,581,comp.os.ms-windows.misc,343,comp.sys.mac.hardware,46.0,59.04%
5,5,386,sci.crypt,373,comp.sys.mac.hardware,1.0,96.63%
6,6,567,comp.graphics,334,comp.sys.mac.hardware,17.0,58.91%
7,7,435,sci.space,409,0,0.0,94.02%
8,8,511,talk.politics.guns,393,talk.religion.misc,37.0,76.91%
9,9,900,rec.motorcycles,101,talk.religion.misc,58.0,11.22%


## 使用提示语对文本进行总结

In [10]:

items_per_cluster = 5
COMPLETIONS_MODEL = "text-davinci-003"

for i in range(num_of_clusters):
    cluster_name = new_df[new_df.cluster == i].iloc[0].rank1
    print(f"Cluster {i}, Rank 1: {cluster_name}, Theme:", end=" ")

    content = "\n".join(
        embedding_df[embedding_df.cluster == i].text.sample(items_per_cluster, random_state=42).values
    )
    response = openai.Completion.create(
        model=COMPLETIONS_MODEL,
        prompt=f'''我们想要给下面的内容，分组成有意义的类别，以便我们可以对其进行总结。请根据下面这些内容的共同点，总结一个50个字以内的新闻组的名称。比如 “PC硬件”\n\n内容:\n"""\n{content}\n"""新闻组名称：''',
        temperature=0,
        max_tokens=100,
        top_p=1,
    )
    print(response["choices"][0]["text"].replace("\n", ""))

Cluster 0, Rank 1: sci.electronics, Theme: 光学感知
Cluster 1, Rank 1: comp.sys.ibm.pc.hardware, Theme: 电脑硬件及其应用
Cluster 2, Rank 1: talk.politics.misc, Theme: 未来科技的潜在影响
Cluster 3, Rank 1: rec.sport.baseball, Theme: 棒球讨论
Cluster 4, Rank 1: comp.os.ms-windows.misc, Theme: PC软件与硬件
Cluster 5, Rank 1: sci.crypt, Theme: 移动安全加密
Cluster 6, Rank 1: comp.graphics, Theme: 计算机图形处理器应用
Cluster 7, Rank 1: sci.space, Theme: 太空新闻与观点
Cluster 8, Rank 1: talk.politics.guns, Theme: 枪支政策热点
Cluster 9, Rank 1: rec.motorcycles, Theme: 谜题讨论
Cluster 10, Rank 1: rec.sport.hockey, Theme: 冰球比赛结果
Cluster 11, Rank 1: comp.windows.x, Theme: X窗口管理器和GUI开发
Cluster 12, Rank 1: talk.politics.mideast, Theme: 历史冲突与政治影响
Cluster 13, Rank 1: sci.med, Theme: 医学与心理学
Cluster 14, Rank 1: comp.os.ms-windows.misc, Theme: 科技产品"""
Cluster 15, Rank 1: rec.autos, Theme: 汽车和摩托车维护
Cluster 16, Rank 1: sci.electronics, Theme: 质量管理与统计
Cluster 17, Rank 1: misc.forsale, Theme: 电子设备交易
Cluster 18, Rank 1: soc.religion.christian, Theme: 宗教信仰与历史
Cluste

In [11]:

items_per_cluster = 1
COMPLETIONS_MODEL = "text-davinci-003"

for i in range(num_of_clusters):
    cluster_name = new_df[new_df.cluster == i].iloc[0].rank1
    print(f"Cluster {i}, Rank 1: {cluster_name}, 抽样翻译:", end=" ")

    content = "\n".join(
        embedding_df[(embedding_df.cluster == i) & (embedding_df.n_tokens > 100)].text.sample(items_per_cluster, random_state=42).values
    )
    response = openai.Completion.create(
        model=COMPLETIONS_MODEL,
        prompt=f'''请把下面的内容翻译成中文\n\n内容:\n"""\n{content}\n"""翻译：''',
        temperature=0,
        max_tokens=2000,
        top_p=1,
    )
    print(response["choices"][0]["text"].replace("\n", ""))


Cluster 0, Rank 1: sci.electronics, 抽样翻译: 我有一台GE Emerson 13英寸的彩色电视机大约有3年了，有一天，当我们正在看某些东西（我怀疑这个节目是不是原因）时，我们听到了一声轻微的爆炸声。我们的屏幕变黑了，但是还有声音，所以我们以为“哦，我们的节目有特殊效果”。但是很快声音也停止了，烟雾开始从电视机的背面冒出来。我们是聪明的电子工程师，我们拔掉电视机的插头，然后打电话给客服，可是一直被转接，最后我们放弃了修理电视机的想法，又买了一台新的（我们也想要一台更大的！）。最后，我想知道的是：我的问题是个案例还是常见的？（我记得读过有关俄罗斯电视机爆炸的报道，但是在美国这里没有）。为什么电视机的显像管会爆炸或者冒烟？我还有剩下的电视机，今年夏天我可能会把它拆开看看。有没有人知道在哪里可以买到这种电视机的零件？（可能花费比电视机还要多）。
Cluster 1, Rank 1: comp.sys.ibm.pc.hardware, 抽样翻译: 由于每年价格的显著下降，我可以买到一台更大、更强大的服务器机器。但是我想使用的链接（ISDN 64K）成本高昂，带宽有限。这就是为什么我感兴趣的是看看这样的链接是否可以使用，看看通过它传输的是什么流量。嗯，你们中心有监控流量吗？你们运行标准的MS Windows程序，比如Word吗？什么样的数据包会穿过它？你们有多大的链接（2Mb或10Mb？）？网络上的平均流量是多少，还是有很多的高峰，然后又有很多的低点？你说的“网络东西”是什么意思？我的想法是使用MS Windows和其他Windows应用程序在网络上运行，这些进程都在服务器上运行，我只是得到显示器。我想知道X和随后的DV / X协议在传输这些图像并在客户机上显示它们时有多好。
Cluster 2, Rank 1: talk.politics.misc, 抽样翻译: 以下内容可以在某个FTP存档中找到，我在这封“大哥大”的恶魔般的备忘录中插入了我的评论：看！这显然是禁止我们自己的螺纹规格的第一步。如果不从头到尾抗争这种疯狂，使用我们无畏领袖“慷慨”定义的螺纹以外的螺纹将是一种犯罪。废话！我说！ANSI标准螺纹会有微妙的弱点，让他们的代理人更容易拆卸我们的汽车，导致我们的消声器在不合适的时候掉下来。哈！“相信我们”看到了吗？！这是第一步。很明显

ValueError: a must be greater than 0 unless no samples are taken

## 利用文本摘要支持无限轮数的聊天


In [12]:

history = """User : 你是谁？
Assistant : 我是一个人工智能语言模型，可以回答各种问题，包括做菜的问题。

User : 请问鱼香肉丝怎么做？
Assistant : 鱼香肉丝的做法：

1. 猪里脊肉切丝，加入盐、淀粉、料酒搅拌均匀，腌制10分钟
2. 胡萝卜、青椒、木耳切丝备用
3. 热锅凉油，放入蒜末、姜末煸炒出香味，加入肉丝翻炒至变色
4. 加入胡萝卜、青椒、木耳翻炒，加入酱油、糖、醋、盐、味精、水淀粉搅拌均匀，倒入锅中翻炒均匀即可。

提示：鱼香肉丝的关键是调料的配比，酱油、糖、醋、盐、味精的比例要适合自己的口味。

User : 那蚝油牛肉呢？
Assistant : 蚝油牛肉的做法：

1. 将牛肉切成薄片，加入盐、淀粉、料酒腌制10分钟
2. 葱姜切末，蒜切片，青红椒切丝备用
3. 热锅凉油，放入葱姜末、蒜片煸炒出香味，加入牛肉翻炒至变色
4. 加入蚝油、酱油、糖、水淀粉调味，加入青红椒翻炒均匀即可。

提示：蚝油牛肉的关键是牛肉要切薄，翻炒时火候要快，保证牛肉口感鲜嫩。调味时，蚝油和酱油的比例也要适合自己的口味。
"""

def summarize(text, max_tokens=200):
    response = openai.Completion.create(
        model=COMPLETIONS_MODEL,
        prompt=text + "\n\n请总结一下上面User和Assistant聊了些什么：\n",
        max_tokens=max_tokens,
    )
    return response["choices"][0]["text"]

summarized = summarize(history)
print(summarized)


User和Assistant聊了如何做鱼香肉丝和蚝油牛肉的步骤，以及各项调料的配比需要符合自己的口味。


In [14]:

class Conversation:
    def __init__(self, prompt, num_of_round):
        self.prompt = prompt
        self.num_of_round = num_of_round
        self.messages = []
        self.messages.append({"role": "system", "content": self.prompt})

    def ask(self, question):
        try:
            self.messages.append({"role": "user", "content": question})
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=self.messages,
                temperature=0.5,
                max_tokens=2048,
                top_p=1,
            )
        except Exception as e:
            print(e)
            return e

        message = response["choices"][0]["message"]["content"]
        self.messages.append({"role": "assistant", "content": message})

        if len(self.messages) > self.num_of_round*2 + 1:
            # Remove the first round conversation left.
            del self.messages[1:3]
        return message
   

prompt = summarized + "\n\n请你根据已经聊了的内容，继续对话："
conversation = Conversation(prompt, 5)

question = "那宫保鸡丁呢？"
answer = conversation.ask(question)
print("User : %s" % question)
print("Assistant : %s\n" % answer)

User : 那宫保鸡丁呢？
Assistant : 宫保鸡丁的做法也比较简单，我们需要准备鸡肉、花生米、青蒜、干辣椒、生姜、蒜末、盐、糖、酱油、料酒和淀粉。

首先，将鸡肉切成小块，加入盐、糖、酱油、料酒和淀粉拌匀腌制10分钟。

然后，将花生米炒香备用。将干辣椒、生姜、蒜末切成小块备用。

接下来，将腌制好的鸡肉放入锅中煸炒至变色，捞出备用。

再将干辣椒、生姜、蒜末放入锅中煸炒出香味，加入鸡肉和花生米一起翻炒，最后加入青蒜炒匀即可。

调料的配比可以根据自己的口味进行调整，让宫保鸡丁更符合自己的口味。

